In [106]:
import json
import csv
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn import preprocessing
import couchdb

In [107]:
# Dict to store all data. sa4 as key.
data_dict = defaultdict(dict)

In [108]:
# data_dict

#### Get Tweet Counts and Sentiment scores for each SA4 (Must be Tunneled into Couchdb instance)

In [109]:
user = 'admin'
password = 'password'
COUCH_ADDRESS = 'localhost'

# Connect to Couch DB Server
# server = couchdb.Server("http://{}:{}@{}:5984/".format(user, password, COUCH_ADDRESS))
server = couchdb.Server("http://{}:{}@{}:15984/".format(user, password, COUCH_ADDRESS))
db = server['tweets']

front_end_db = server['front_end']

In [110]:
tweet_counts = {}
sent_sum = {}
# Store Tweet Counts
for code in db.view('Results/TweetCount', group='true'):
    tweet_counts[code.key] = code.value

# Store Sentiment Scores
for code in db.view('Results/SentimentSum', group='true'):
    sent_sum[code.key] = code.value

In [111]:
for key in tweet_counts.keys():
    count = tweet_counts[key]
    sent = sent_sum[key]
    score = sent/count
    
    # store in data dict
    data_dict[key]['sentiment_score'] = score
    data_dict[key]['tweet_counts'] = count
    data_dict[key]['sent_sum'] = sent
    

#### Process data from Aurin & SA4 Geojson 

In [112]:
# Import sa4 geojson file and aurin data files
sa4_geo_file = json.load(open('SA4_geojson.json'))
sa4_centroids = json.load(open('sa4_geojson_centroid.json'))


# crime data needs lga_sa4 conversion
crime_data = json.load(open('crimedata.json'))
income_data = json.load(open('equivalisedincomedata.json'))
family_data = json.load(open('familycommunitydata.json'))
unemployment_data = json.load(open('SA4_unemployment.json'))
population_data = json.load(open('populationdata.json'))
industry_data = json.load(open('industry.json'))
socio_advantage_data = json.load(open('socioirsaddata.json'))
personal_income = json.load(open('personalincomedata.json'))
house_price = json.load(open('housepricedata.json'))

lga_sa4 = open('lga_sa4.csv', 'r')

In [113]:
# Process sa4 lga conversion
lga_sa4_dict = defaultdict(list)
for line in lga_sa4:
    line = line.strip('\n')
    (lga, sa4) = line.split(",")
    lga_sa4_dict[lga] = sa4
    
# Create DF for data analysis


In [114]:
def process_aurin(data_set, features):

    for feature in features:
        try:
            for item in data_set['features']:
                if item['properties']['sa4_code_2016'] in data_dict.keys():
                    data_dict[item['properties']['sa4_code_2016']][feature] = item['properties'][feature]
        except:
            continue

In [115]:
def add_to_dict(key, d, val):
    if key in d:
        d[key] = d[key] + val
    else:
        d[key] = val

In [116]:
# Process crime sa4_lga conversion
# Add sa4 to each LGA in crime_data
# for item in crime_data['features']:
#     if item['properties']['lga_code'] in lga_sa4_dict.keys():
        
#         a = item['properties']['total_division_a_offences']
#         b = item['properties']['total_division_b_offences']
#         c = item['properties']['total_division_c_offences']
#         d = item['properties']['total_division_d_offences']
#         e = item['properties']['total_division_e_offences']
#         f = item['properties']['total_division_f_offences']
#         sum_crimes = a+b+c+d+e+f
        
#         item['properties']['sa4_code_2016'] = lga_sa4_dict[item['properties']['lga_code']]
#         item['properties']['sum_crimes'] = sum_crimes

In [117]:
# Process for Unemployment job search weeks data
for item in unemployment_data['features']:
    if str(item['properties']['sa4_code']) in data_dict.keys():
        data_dict[str(item['properties']['sa4_code'])]['unemployed_rate'] = item['properties']['unemployed_tot_000']
#         data_dict[str(item['properties']['sa4_code'])]['avg_duration_job_search_wks'] = item['properties']['avg_duration_job_search_wks']

In [118]:
# Median House Price
for item in house_price['features']:
    if str(item['properties']['sa42016code']) in data_dict.keys():
        data_dict[str(item['properties']['sa42016code'])]['median_house_price'] = item['properties']['sold_both_auction_private_treaty_medianprice']

In [119]:
# Process income and family features
income_features = ['equivalised_total_household_income_census_median_weekly']
family_features = ['rent_mortgage_payments_census_average_monthly_household_payment', 'sa4_name_2016']
# crime_features = ['total_division_a_offences', 'total_division_b_offences', 'total_division_c_offences', 
#                   'total_division_d_offences', 'total_division_e_offences', 'total_division_f_offences', 
#                   'sum_crimes']
# socio_features = ['irsad_score']
personal_income_features = ['mean_aud', 'median_aud']

process_aurin(income_data, income_features)
process_aurin(family_data, family_features)
process_aurin(personal_income, personal_income_features)
# process_aurin(crime_data, crime_features)
# process_aurin(socio_advantage_data, socio_features)

In [120]:
data_dict

defaultdict(dict,
            {'101': {'sentiment_score': 0.2846828662370934,
              'tweet_counts': 38643,
              'sent_sum': 11001,
              'unemployed_rate': 5.76200061,
              'median_house_price': 550000.0,
              'equivalised_total_household_income_census_median_weekly': 795,
              'rent_mortgage_payments_census_average_monthly_household_payment': 1805,
              'sa4_name_2016': 'Capital Region',
              'mean_aud': 54054,
              'median_aud': 44110},
             '102': {'sentiment_score': 0.008370183196462414,
              'tweet_counts': 18996,
              'sent_sum': 159,
              'unemployed_rate': 9.20597806,
              'median_house_price': 680000.0,
              'equivalised_total_household_income_census_median_weekly': 774,
              'rent_mortgage_payments_census_average_monthly_household_payment': 1890,
              'sa4_name_2016': 'Central Coast',
              'mean_aud': 53716,
           

In [121]:
# # Add population data
# for item in population_data['features']:
#     if item['properties']['sa4_code16'] in data_dict.keys():
#         data_dict[item['properties']['sa4_code16']]['persons_total'] = item['properties']['persons_total']


In [122]:
# Put centroid in
for item in sa4_centroids['features']:
    if item['properties']['SA4_CODE16'] in data_dict.keys():
        data_dict[item['properties']['SA4_CODE16']]['centroid'] = item['geometry']['coordinates']
    

In [123]:
# for item in industry_data['features']:
#     if item['properties']['sa4_code_2016'] in data_dict.keys():
#         data_dict[item['properties']['sa4_code_2016']]['num_recreation_busi'] = item['properties']['number_businesses_industry_30_june_arts_recreation_services_num']
#         data_dict[item['properties']['sa4_code_2016']]['num_scientific_busi'] = item['properties']['nmbr_bsnsss_indstry_30_jne_prfssnl_scntfc_tchncl_srvcs_nm']
#         data_dict[item['properties']['sa4_code_2016']]['num_mining_busi'] = item['properties']['number_of_businesses_by_industry_as_at_30_june_mining_num']
#         data_dict[item['properties']['sa4_code_2016']]['num_finance_busi'] = item['properties']['nmbr_bsnsss_indstry_30_jne_fnncl_insrnce_srvcs_nm']
#         data_dict[item['properties']['sa4_code_2016']]['num_agri_busi'] = item['properties']['nmbr_bsnsss_indstry_30_jne_agrcltre_frstry_fshng_nm']

In [124]:
# personal_income

#### Analysis of Tweets

In [125]:
data_df=pd.DataFrame.from_dict(data_dict, orient='index')

In [126]:
data_df

,sentiment_score,tweet_counts,sent_sum,unemployed_rate,median_house_price,equivalised_total_household_income_census_median_weekly,rent_mortgage_payments_census_average_monthly_household_payment,sa4_name_2016,mean_aud,median_aud,centroid
101,0.284683,38643,11001,5.762001,550000.0,795,1805,Capital Region,54054,44110,"[149.24, -35.56]"
102,0.008370,18996,159,9.205978,680000.0,774,1890,Central Coast,53716,43677,"[151.29, -33.31]"
103,0.361404,855,309,5.622973,429000.0,742,1625,Central West,54112,43834,"[148.36, -33.22]"
104,0.246459,353,87,2.280920,516000.0,648,1568,Coffs Harbour - Grafton,45249,37017,"[152.77, -29.82]"
106,0.827586,812,672,7.075872,485000.0,786,1801,Hunter Valley exc Newcastle,58769,45556,"[150.98, -32.35]"
...,...,...,...,...,...,...,...,...,...,...,...
603,0.358566,251,90,0.854961,400000.0,625,1277,South East,44407,36927,"[146.82, -42.61]"
604,0.482456,114,55,3.025266,305000.0,644,1271,West and North West,49216,41816,"[145.56, -41.73]"
701,0.106405,7697,819,5.404876,470000.0,1316,2383,Darwin,71185,60572,"[131.11, -12.53]"
702,0.862069,29,25,3.117814,420000.0,954,1945,Northern Territory - Outback,59232,51797,"[133.37, -19.49]"


In [127]:
data_df.corr()

,sentiment_score,tweet_counts,sent_sum,unemployed_rate,median_house_price,equivalised_total_household_income_census_median_weekly,rent_mortgage_payments_census_average_monthly_household_payment,mean_aud,median_aud
sentiment_score,1.000000,-0.036450,0.023119,-0.072039,-0.060068,-0.058141,-0.087230,0.014604,0.002779
tweet_counts,-0.036450,1.000000,0.991297,0.293492,0.165644,0.293118,0.213288,0.309928,0.263137
sent_sum,0.023119,0.991297,1.000000,0.298804,0.168684,0.281039,0.211594,0.308161,0.248343
unemployed_rate,-0.072039,0.293492,0.298804,1.000000,0.250462,0.195451,0.311829,0.106939,0.188333
median_house_price,-0.060068,0.165644,0.168684,0.250462,1.000000,0.760515,0.843569,0.749251,0.565756
equivalised_total_household_income_census_median_weekly,-0.058141,0.293118,0.281039,0.195451,0.760515,1.000000,0.904217,0.922534,0.921939
rent_mortgage_payments_census_average_monthly_household_payment,-0.087230,0.213288,0.211594,0.311829,0.843569,0.904217,1.000000,0.845762,0.802814
mean_aud,0.014604,0.309928,0.308161,0.106939,0.749251,0.922534,0.845762,1.000000,0.885472
median_aud,0.002779,0.263137,0.248343,0.188333,0.565756,0.921939,0.802814,0.885472,1.000000


In [128]:
# import seaborn as sns; sns.set_theme(color_codes=True)


# ax = sns.regplot(x="mean_aud", y="sentiment_score", data=data_df)

In [132]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 100)) 

sentiment_score_scaled = min_max_scaler.fit_transform(np.array(data_df.sentiment_score).reshape(-1, 1))
tweet_counts_scaled = min_max_scaler.fit_transform(np.array(data_df.tweet_counts).reshape(-1, 1))
sent_sum_scaled = min_max_scaler.fit_transform(np.array(data_df.sent_sum).reshape(-1, 1))
unemployed_rate_scaled = min_max_scaler.fit_transform(np.array(data_df.unemployed_rate).reshape(-1, 1))
median_house_price_scaled = min_max_scaler.fit_transform(np.array(data_df.median_house_price).reshape(-1, 1))
equivalised_total_household_income_census_median_weekly_scaled = min_max_scaler.fit_transform(np.array(data_df.equivalised_total_household_income_census_median_weekly).reshape(-1, 1))
rent_mortgage_payments_census_average_monthly_household_payment_scaled = min_max_scaler.fit_transform(np.array(data_df.rent_mortgage_payments_census_average_monthly_household_payment).reshape(-1, 1))
mean_aud_scaled = min_max_scaler.fit_transform(np.array(data_df.mean_aud).reshape(-1, 1))
median_aud_scaled = min_max_scaler.fit_transform(np.array(data_df.median_aud).reshape(-1, 1))

data_df['sentiment_score_scaled'] = sentiment_score_scaled
data_df['tweet_counts_s'] = tweet_counts_scaled
data_df['sent_sum_s'] = sent_sum_scaled
data_df['unemployed_rate_scaled'] = unemployed_rate_scaled
data_df['median_house_price_s'] = median_house_price_scaled
data_df['equivalised_total_household_income_census_median_weekly_s'] = equivalised_total_household_income_census_median_weekly_scaled
data_df['rent_mortgage_payments_census_average_monthly_household_payment_s'] = rent_mortgage_payments_census_average_monthly_household_payment_scaled
data_df['mean_aud_s'] = mean_aud_scaled
data_df['median_aud_s'] = median_aud_scaled


for index, row in data_df.iterrows():
    if str(index) in data_dict.keys():
        data_dict[str(index)]['sentiment_score'] = [data_dict[str(index)]['sentiment_score'], row['sentiment_score_scaled']]
        data_dict[str(index)]['tweet_counts'] = [data_dict[str(index)]['tweet_counts'], row['tweet_counts_s']]
        data_dict[str(index)]['sent_sum'] = [data_dict[str(index)]['sent_sum'], row['sent_sum_s']]
        data_dict[str(index)]['median_house_price'] = [data_dict[str(index)]['median_house_price'], row['median_house_price_s']]
        data_dict[str(index)]['equivalised_total_household_income_census_median_weekly'] = [data_dict[str(index)]['equivalised_total_household_income_census_median_weekly'], row['equivalised_total_household_income_census_median_weekly_s']]
        data_dict[str(index)]['rent_mortgage_payments_census_average_monthly_household_payment'] = [data_dict[str(index)]['rent_mortgage_payments_census_average_monthly_household_payment'], row['rent_mortgage_payments_census_average_monthly_household_payment_s']]
        data_dict[str(index)]['mean_aud'] = [data_dict[str(index)]['mean_aud'], row['mean_aud_s']]
        data_dict[str(index)]['median_aud'] = [data_dict[str(index)]['median_aud'], row['median_aud_s']]
        try:
            data_dict[str(index)]['unemployed_rate'] = [data_dict[str(index)]['unemployed_rate'], row['unemployed_rate_scaled']]
        except KeyError:
            data_dict[str(index)]['unemployed_rate'] = ['No Record', 'No Record']
# data_dict


In [133]:
data_dict

defaultdict(dict,
            {'101': {'sentiment_score': [0.2846828662370934,
               17.437397027490963],
              'tweet_counts': [38643, 10.49293040104055],
              'sent_sum': [11001, 5.7758696769437625],
              'unemployed_rate': [5.76200061, 13.191104166666163],
              'median_house_price': [550000.0, 10.821643286573147],
              'equivalised_total_household_income_census_median_weekly': [795,
               22.285714285714292],
              'rent_mortgage_payments_census_average_monthly_household_payment': [1805,
               33.54614045269878],
              'sa4_name_2016': 'Capital Region',
              'mean_aud': [54054, 16.168338796728577],
              'median_aud': [44110, 31.582840236686394],
              'centroid': [149.24, -35.56]},
             '102': {'sentiment_score': [0.008370183196462414,
               11.297115182143608],
              'tweet_counts': [18996, 5.157942471087552],
              'sent_sum': [159, 0.14

In [134]:
out = []
for key, value in data_dict.items():
    curr_d = value
    curr_d['sa4_code'] = key
    out.append(curr_d)

In [135]:
out

[{'sentiment_score': [0.2846828662370934, 17.437397027490963],
  'tweet_counts': [38643, 10.49293040104055],
  'sent_sum': [11001, 5.7758696769437625],
  'unemployed_rate': [5.76200061, 13.191104166666163],
  'median_house_price': [550000.0, 10.821643286573147],
  'equivalised_total_household_income_census_median_weekly': [795,
   22.285714285714292],
  'rent_mortgage_payments_census_average_monthly_household_payment': [1805,
   33.54614045269878],
  'sa4_name_2016': 'Capital Region',
  'mean_aud': [54054, 16.168338796728577],
  'median_aud': [44110, 31.582840236686394],
  'centroid': [149.24, -35.56],
  'sa4_code': '101'},
 {'sentiment_score': [0.008370183196462414, 11.297115182143608],
  'tweet_counts': [18996, 5.157942471087552],
  'sent_sum': [159, 0.14743366782779332],
  'unemployed_rate': [9.20597806, 22.449204303959792],
  'median_house_price': [680000.0, 16.032064128256515],
  'equivalised_total_household_income_census_median_weekly': [774,
   19.885714285714286],
  'rent_mortg

### Update Sentiment Scores periodically

In [120]:

#get file from DB
in_file = front_end_db.get_attachment('output', 'out_data.json').read()
in_json = json.loads(in_file.decode('utf8').replace("'", '"'))


In [121]:
in_json[0]

{'sentiment_score': [0.2846828662370934, 0.17437397027490964],
 'tweet_counts': [38643, 0.10564302488661186],
 'sent_sum': [11001, 0.05829342666429148],
 'median_house_price': [550000.0, 0.10821643286573147],
 'equivalised_total_household_income_census_median_weekly': [795,
  0.22285714285714286],
 'rent_mortgage_payments_census_average_monthly_household_payment': [1805,
  0.3354614045269878],
 'sa4_name_2016': 'Capital Region',
 'mean_aud': [54054, 0.16168338796728587],
 'median_aud': [44110, 0.31582840236686405],
 'centroid': [149.24, -35.56],
 'sa4_code': '101'}

In [122]:
# Calculate sentiment score and normalised sentiment score
sentiment_score = {}
for key in tweet_counts.keys():
    count = tweet_counts[key]
    sent = sent_sum[key]
    score = sent/count
    
    sentiment_score[key] = score


In [123]:
# sentiment_score

In [124]:
# Update file
for item in in_json:
    if item['sa4_code'] in sentiment_score.keys():
        key = item['sa4_code']
        item['sentiment_score'] = sentiment_score[key]
        item['sent_sum'] = sent_sum[key]
        item['tweet_counts'] = tweet_counts[key]

In [125]:
# Put attachment to DB
doc = front_end_db['test_3']
front_end_db.put_attachment(doc, in_json, 'out_data.json', "application/json")

In [136]:
# Put attachment to DB
# doc = front_end_db['output']
# front_end_db.put_attachment(doc, out, 'out_data_0_100.json', "application/json")

#### Process Output json

In [ ]:
# Output processed geojson
def merge_data(data, sa4_geo):
    output={"type": "FeatureCollection", "features":[]}
    for row in sa4_geo['features']:
        key = row['properties']['SA4_CODE16']
        if key in data.keys():
            for feature in data[key].keys():
                row['properties'][feature]= data[key][feature]
        else:
            for feature in data[key].keys():
                row['properties'][feature]='No Record'
   
    with open('output.json', 'w') as outfile:
        json.dump(sa4_geo, outfile)

In [ ]:
# merge_data(data_dict, sa4_geo_file)


In [ ]:
# check = json.load(open('output.json'))
# for item in check['features']:
#     if item['properties']['SA4_CODE16'] == '117':
#         print(item)

In [ ]:
len(data_dict)

In [ ]:
data_dict.keys()

In [ ]:
sa4_geo_file['features'][2]